In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd 

In [3]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("강남구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5  # 최대 페이지 수 설정

while page_number <= MAX_PAGES:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:  
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

# CSV 파일 저장
file_name = "Gangnam-gu_restaurants.csv"
df = pd.DataFrame(results)
df.to_csv(file_name, index=False, encoding="utf-8-sig")  
print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 육달집 강남본점
2. 밤별 포차 라운지 강남역본점
3. 영천영화 청담점
4. 치스타리에 강남역점
5. 동어동락 삼성본점
6. 미도인 강남
7. 임피리얼팰리스서울 카페 델마르
8. 을지다락 강남
9. 사보텐 아워홈빌딩점
10. 자매수산
11. 다몽집 신논현본점
12. 쭈꾸미랩소디 강남점
13. 샤블리 신사본점
14. 고기꾼김춘배 강남점
15. 골드피쉬 딤섬퀴진
16. 일품헌 역삼점
17. 어거스트 힐 강남점
18. 심퍼티쿠시 신사점
19. 노티드 스튜디오 청담
20. 논현 에이엠포차
21. 삼성골목집 삼성본점
22. 청담 마코토
23. 강삼가든
24. 일두한우정육식당
25. 세광양대창 논현점
26. 일일향 압구정1호점
27. 고메램 삼성점
28. 남도애찬 강남본점
29. 울프강 스테이크하우스
30. 스시유 삼성본점
31. 봉밀가 강남구청점
32. 테라스룸
33. 상무초밥 강남점
34. 오레노카츠 선릉점
35. 은행골 신사점
36. 텍사스데브라질 압구정점
37. 봉우화로
38. 그랑디르
39. 까치화방 강남역점
40. 흑우정 압구정로데오 본점
41. 단골손님 강남1호점
42. 스시산원 반주헌
43. 갓포 모로미 도산공원점
44. 그랑씨엘
45. 은비갈비 신사본점
46. 자연산 해담일식 대게마을
47. 제주옥탑 블랙BBQ
48. 보물섬
49. 쮸즈
50. 뜨락
51. 무월식탁
52. 클랩피자 청담
53. 팀호완 삼성점
54. 임고집한우 본점
55. 케르반 코엑스몰점
56. 봉우이층집
57. 만카이 압구정점
58. 트라가 역삼점
59. 강남보물섬
60. 마르케
61. 국고집
62. 장인닭갈비 강남점
63. 문화시민 서울 뱅뱅사거리점
64. 츄라우미 역삼본점
65. 까사생갈비
66. 아리양
67. 한성양꼬치 선릉2호점
68. 명우돈가
69. 카페 공명 신사 가로수길점
70. 빽돈 신논현직영점
71. 김돈이 본점
72. 농민백암순대 본점
73. 을지다락 도산공원
74. 개성집
75. 보노보노 삼성점
76. 하남돼지집 